# Imports

In [3]:
%matplotlib inline
# Imports
from skimage import io
from skimage.color import rgb2gray
from skimage.filters import median
from skimage.feature import blob_log
from skimage.draw import disk
import os, math, csv
import matplotlib.pyplot as plt
import numpy as np
from numba import njit

# Variables

In [ ]:
# CSV file name
CSV = "Results.csv"

# Center of the detection circle
CENTER = (1032, 1384)

# Radius of the detection circle
RADIUS = 900

# Minimal distance of the blobs from the circle border
BORDER_DISTANCE = 15

# Lower for smaller dots
MIN_SIZE = 1.5

# Higher for bigger dots
MAX_SIZE = 4

# Higher for increased precision
LAYERS = 10

# Lower to also detect darker dots
THRESHOLD = .005

# Allowed overlap for dots
OVERLAP = 0.35

# Blurring radius
BLURRING = (4, 4)

# Number of used threads
MAX_THREADS = 16

# ProcessPools do not function correctly inside Jupyter Notebooks
USE_CONCURRENCY = False

# Folder to load the images from
img_path = os.path.join(os.getcwd(), "Images")

# Folder to save the analysis results to
save_path = os.path.join(os.getcwd(), "Results")

# Code

In [ ]:
# Check if paths actually exist, else create them
os.makedirs(img_path, exist_ok=True)
os.makedirs(save_path, exist_ok=True)

# Load images
images = []
rows = []
for t in os.walk(img_path):
    for file in t[2]:
        images.append(os.path.join(t[0], file))

def analyse_image(path: str) -> Tuple[str, str, int]:
    """
    Function to analyse the given image

    :param path: The path leading to the image
    :return: The name of the image, the path leading to the image and the counted cells as tuple
    """
    # Load image
    image = rgb2gray(io.imread(path))
    # Calculate image average
    av = np.average(image)
    # Create mask filled with average
    mask = np.full(fill_value=av, shape=image.shape, dtype="uint8")
    # Mask image
    rr, cc = circle(CENTER[1], CENTER[0], RADIUS)
    mask[cc, rr] = image[cc, rr]
    # Create result image
    fig = plt.figure(figsize=(30, 30))
    ax = fig.add_subplot(1, 1, 1)
    plt.imshow(mask, cmap="gray")

    @njit(cache=True)
    def eu_dist(p1: Tuple[int, int], p2: Tuple[int, int]) -> float:
        """
        Function to calculate the euclidean distance between p1 and p2

        :param p1: The first point (y:x)
        :param p2: The second point (y:x)
        :return: The euclidean distance as float
        """
        return math.sqrt(((p2[0] - p1[0])**2) + ((p2[1] - p1[1])**2))

    # Detect blobs using Laplacian Of Gaussian blob detection
    blobs = blob_log(image=median(mask, selem=np.ones(shape=BLURRING)),
                     min_sigma=MIN_SIZE, max_sigma=MAX_SIZE,
                     num_sigma=LAYERS, threshold=THRESHOLD,
                     exclude_border=False, overlap=OVERLAP)
    count = 0
    # Draw blobs
    for blob in blobs:
        y, x, r = blob
        if eu_dist((y, x), CENTER) <= RADIUS - BORDER_DISTANCE:
            c = plt.Circle((x, y), r, color="red", linewidth=0.85, fill=False)
            ax.add_patch(c)
            count += 1
    # Get file name
    name = os.path.basename(path)
    # Save figure
    ax.set_title(f"Counting for {name}\nCount: {count}")
    fig.savefig(os.path.join(save_path, f"{name}_counting_result.png"), bbox_inches="tight")
    plt.close(fig)
    return name, path, count

if __name__ == "__main__":
    # Open a new CSV file in write mode
    with open(CSV, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=";",
                            quotechar="|", quoting=csv.QUOTE_MINIMAL)
        # Write the table header to the file
        writer.writerow(("Filename", "Path", "Count"))
        print(f"{'Filename':^30}{'Count':^30}")
        if USE_CONCURRENCY:
            # Use concurrency to spead up execution time
            with ProcessPoolExecutor(max_workers=MAX_THREADS) as e:
                res = e.map(analyse_image, images)
                for row in res:
                    print(f"{row[0]:^30}{row[2]:^30}")
                    # Write the rows to the opened csv file
                    writer.writerow(row)
        else:
            for path in images:
                row = analyse_image(path)
                writer.writerow(row)